In [54]:
# Imports
import numpy as np
import pandas as pd

import os
os.getcwd()

'/home/peisenha/ownCloud/office/workspace/software/repositories/organizations/restudToolbox/package/development/working'

## Simulation

I will now manually check the simulation of some samples to see if the problem is in the simulation.

In [134]:
df = pd.read_csv('data.respy.dat', delim_whitespace=True, header=0, na_values='.')
df.set_index(['Identifier', 'Period'], drop=False, inplace=True)
df.columns

Index(['Identifier', 'Period', 'Choice', 'Wage', 'Experience_A',
       'Experience_B', 'Years_Schooling', 'Lagged_Activity', 'Type',
       'Total_Reward_1', 'Total_Reward_2', 'Total_Reward_3', 'Total_Reward_4',
       'Systematic_Reward_1', 'Systematic_Reward_2', 'Systematic_Reward_3',
       'Systematic_Reward_4', 'Shock_Reward_1', 'Shock_Reward_2',
       'Shock_Reward_3', 'Shock_Reward_4', 'Discount_Rate', 'General_Reward_1',
       'General_Reward_2', 'Common_Reward', 'Wage_Internal'],
      dtype='object')

In [136]:

cond = df['Choice'] == 2
#wage_sys = df['Wage_Internal']

col_1 = df['Wage_Internal'] + df['Common_Reward'] + df['General_Reward_2']
col_2 = df['Total_Reward_2']
np.testing.assert_array_almost_equal(col_1.loc[cond], col_2.loc[cond])

